<a href="https://colab.research.google.com/github/JorgeMoranBornemann/BScThesis/blob/main/Notebook_BScThesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Permafrost thaw data analysis**

The purpose of this notebook is to run through the steps, and explain, how the data analysis was performed.

It will be divided into several parts.
1. Importing data, libraries, dependencies
2. SOC - Thaw depth regressions
3. Himidity - Thaw depth regressions
4. TDD pre-processing
5. TTD - thaw depth regressions
6. Impact of SOC & Humidity on TTD regression
7. Impact of rainfall on TTD regression



In [1]:
#Importing required libraries
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import datetime as dt


In [2]:
# importing the datasets and converting them to pandas dataframes

# You need to be a collaborator in GitHub to be able to run this notebook

# URLs for the datasets
URL_soilmoisture = 'https://raw.githubusercontent.com/JorgeMoranBornemann/BScThesis/main/Datasets/all_soilmoisture.csv?token=GHSAT0AAAAAACKOMOO6L6OGFCC5XQBSXAIUZKYP2SA'
URL_ibuttons_overview = 'https://raw.githubusercontent.com/JorgeMoranBornemann/BScThesis/main/Datasets/ibuttons_overview.csv?token=GHSAT0AAAAAACKOMOO6UGAFUQOUHYNL2ADMZKYP3CA'
URL_SoilTParsed = 'https://raw.githubusercontent.com/JorgeMoranBornemann/BScThesis/main/Datasets/SoilTParsed.csv?token=GHSAT0AAAAAACKOMOO6ATXSESCSISR6USL4ZKYP3OA'
URL_all_thawdepth = 'https://raw.githubusercontent.com/JorgeMoranBornemann/BScThesis/main/Datasets/all_thawdepth.csv?token=GHSAT0AAAAAACKOMOO7O7JA24GSOH7XRITIZKYP33Q'
URL_irrigationlog = 'https://raw.githubusercontent.com/JorgeMoranBornemann/BScThesis/main/Datasets/irrigationlog.csv?token=GHSAT0AAAAAACKOMOO7GDXPA23H36DK2APMZKYP4EQ'
URL_plotcoordinates = 'https://raw.githubusercontent.com/JorgeMoranBornemann/BScThesis/main/Datasets/plot_coordinates.csv?token=GHSAT0AAAAAACKOMOO7BPSMAMJHXYOHV3GSZKYP4PA'

# loading the CSV data as pandas df
df_sm = pd.read_csv(URL_soilmoisture)
df_ib = pd.read_csv(URL_ibuttons_overview)
df_sT = pd.read_csv(URL_SoilTParsed)
df_tD = pd.read_csv(URL_all_thawdepth)
df_ir = pd.read_csv(URL_irrigationlog)
df_pl = pd.read_csv(URL_plotcoordinates)


HTTPError: ignored

In [ ]:
print(SM)

     location treatment  pair plot       date  DOY  year  cumLS2022  \
0           1         C     1  1C1  7/21/2022  202  2022          0   
1           1         I     1  1I1  7/21/2022  202  2022          0   
2           1         C     2  1C2  7/21/2022  202  2022          0   
3           1         I     2  1I2  7/21/2022  202  2022          0   
4           1         C     3  1C3  7/21/2022  202  2022          0   
..        ...       ...   ...  ...        ...  ...   ...        ...   
751         5         c     2  5C2  7/28/2022  209  2022         10   
752         5         i     3  5I3  7/28/2022  209  2022         10   
753         5         c     3  5C3  7/28/2022  209  2022         10   
754         5         c     6  5C6  7/28/2022  209  2022         10   
755         5         i     6  5I6  7/28/2022  209  2022         10   

     cumES2023  cumLS2023  ...     B     C     D     E     F     G     H  \
0            0        0.0  ...    31  26.3  21.8  20.8  21.5  21.2  25.

**4. TTD-preprocessing**

In the following section, we extract the number of thawing degree days (TTD) in consecutive 10-day periods from the SoilTParsed.csv dataset. We also extract the thaw depth at the end of this 10-day period. After this, we will substract each subsequent thaw depth to get the thaw depth difference after each 10-day period, ΔTD. This data will then be stored in a separate dataset.

In [ ]:
for col in df_sT.columns:
  print(col)

Unnamed: 0
Time
Temp
Plot
Depth
Location
Day
Month
Year
Hour
Min
TimeParsed
Treatment


In [ ]:
#adds a DOY column to the dataframe
df_sT['Date-conv'] = pd.to_datetime(df_sT[['Year', 'Month', 'Day']])
df_sT['DOY'] = df_sT['Date-conv'].dt.dayofyear


df_sT.loc[['DOY']]








KeyError: ignored

In [ ]:
for col in df_sT.columns:
  print(col)

#df_sT_Day = df_sT.loc[:, ("Day", "Temp")]
#df_sT_Day.head

Unnamed: 0
Time
Temp
Plot
Depth
Location
Day
Month
Year
Hour
Min
TimeParsed
Treatment
Date-conv
DOY


In [ ]:
df_sT.head

<bound method NDFrame.head of         Unnamed: 0               Time  Temp  Plot Depth  Location  Day  Month  \
0                1  21/07/22 21:31:00   9.0   1C1    20         1   21      7   
1                2  22/07/22 01:46:00   8.0   1C1    20         1   22      7   
2                3  22/07/22 06:01:00   8.0   1C1    20         1   22      7   
3                4  22/07/22 10:16:00   7.5   1C1    20         1   22      7   
4                5  22/07/22 14:31:00   7.5   1C1    20         1   22      7   
...            ...                ...   ...   ...   ...       ...  ...    ...   
110452      110453   31/07/2023 20:46   8.0  5LS3     5         5   31      7   
110453      110454    01/08/2023 1:01   7.5  5LS3     5         5    1      8   
110454      110455    01/08/2023 5:16   7.0  5LS3     5         5    1      8   
110455      110456    01/08/2023 9:31   7.5  5LS3     5         5    1      8   
110456      110457   01/08/2023 13:46   8.0  5LS3     5         5    1      8  